<a href="https://colab.research.google.com/github/nischal1234/DS7010/blob/colab/FinalcodeV6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google drive link

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Library install**

In [ ]:
!pip install soundfile

# **Import library**

In [ ]:
import sys
print(sys.version)
import pandas as pd
import numpy as np
import librosa
import librosa.display
import soundfile
import matplotlib.pyplot as plt

# Metafile read

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/DS7010/iemocap_final.csv')
data.head()


# **Path_finding_for_audio**

In [ ]:
def path_finding_for_audio(file_name,session_number):
    #folder_name=file_name
    wav_file=file_name + ".wav"
    number=session_number
    #as per colab
    #sampele location is /content/drive/MyDrive/DS7010/IEMOCAP_full_release
    dataset_directory = '/content/drive/MyDrive/DS7010/IEMOCAP_full_release'
    sessions_to_play = ['Session1','Session2','Session3','Session4','Session5']
    folder_name=extract_folder_name(file_name)
    audio_folder = os.path.join(dataset_directory,sessions_to_play[session_number-1],'sentences','wav',folder_name,wav_file)
    #check_exist(audio_folder)
    return audio_folder

def extract_folder_name(file_name):
    # Find the index of the underscore before the last character
    last_underscore_index = file_name.rfind('_')

    # Extract the substring from the beginning of the string up to the last underscore
    folder_name = file_name[:last_underscore_index]

    return folder_name

def check_exist(audio_folder):
    if os.path.exists(audio_folder):
        print('file exist')
    else:
        print('not exist')

path_finding_for_audio(data.wav_file[4],1)

# **Technique One (MFCCs)**

In [ ]:
import numpy as np
import librosa
from tqdm import tqdm

# Example feature extraction function
def extract_features(audio_signal, sampling_rate):
      # Normalizing time domain signal:
      audio_signal = audio_signal / np.max(np.abs(audio_signal))
      # MFCC configuration:
      N_MFCC = 20
      N_MELS = 60
      WINDOW_LENGTH = int(0.025 * sampling_rate)  # To obtain 25 ms window
      HOP_LENGTH = int(0.010 * sampling_rate)  # 10 ms shift between consecutive windows

      # Extracting MFCCs:
      mfccs = librosa.feature.mfcc(y=audio_signal, sr=sampling_rate, n_mfcc=N_MFCC, n_mels=N_MELS, n_fft=WINDOW_LENGTH, hop_length=HOP_LENGTH).T

      # Feature normalization:
      mfccs -= np.mean(mfccs, axis=0)
      mfccs /= np.std(mfccs, ddof=0, axis=0)
      return mfccs

# Assuming you have a function `path_finding_for_audio()` that returns the file path
def path_finding_for_audio_local(wav_file, session):
    # Your code to find the file path
    file_path=path_finding_for_audio(wav_file,session)
    return file_path

# Initialize lists to store extracted features
all_chroma_stft = []
all_rmse = []
all_spec_cent = []
all_spec_bw = []
all_rolloff = []
all_zcr = []
all_mfcc = []
metadata=data
# Iterate over each row in the metadata and extract features
for index_num, row in tqdm(metadata.iterrows(), total=len(metadata)):
    # Find the file path for the audio
    file_path = path_finding_for_audio_local(row['wav_file'], row['session'])
    # Load audio signal and sampling rate
    audio_signal, sampling_rate = soundfile.read(file_path)
    # Extract features
    mfcc = extract_features(audio_signal, sampling_rate)
    # Pad or truncate feature vectors to match the maximum length
    #max_len = max(len(chroma_stft[0]), 1, len(spec_cent), len(spec_bw), len(rolloff), len(zcr), mfcc.shape[1])

    all_mfcc.append(mfcc)


    #all_mfcc = np.array(all_mfcc)

# Now, you have arrays with consistent feature vector lengths that you can use for further processing.


# **Technique Two**
*this runs perfectly *

In [1]:
from tqdm import tqdm
newdata=[]
def path_finding_for_audio_local(wav_file, session):
    # Your code to find the file path
    file_path=path_finding_for_audio(wav_file,session)
    return file_path

extracted_features=[]
metadata=data
# Iterate over each row in the metadata and extract features
for index_num, row in tqdm(metadata.iterrows(), total=len(metadata)):
    # Find the file path for the audio
    file_path = path_finding_for_audio_local(row['wav_file'], row['session'])
    # Load audio signal and sampling rate
    y, sr = soundfile.read(file_path)
    #print(sr)

    # Extract features
    newdata.append(y)

In [ ]:
from tqdm import tqdm
import librosa
#from scipy.signal import mfcc
Xmfcc = []

for i in tqdm(range(len(newdata))):
    mfcc_features = librosa.feature.mfcc(y=newdata[i], sr=16000)
    #mfcc_features = mfcc(data[i], 16000, nfft=512)
    mfcc_resized = np.resize(mfcc_features, (size, 1472)) # need to change the value of 1472
    Xmfcc.append(mfcc_resized)

Xmfccs = np.array(Xmfcc)